In [3]:
# Alternate GUI for IAFautoclassification script using widgets from Jupyter
# Written by: Robert Granat, Jan-Feb 2022.

# Imports
import ipywidgets as widgets
import os
import sys
import src.IAFautoclass.SqlHelper.IAFSqlHelper as sql
import src.IAFautoclass.IAFautomaticClassiphyer

# Class definition for the GUI
class IAFautoclass_GUI:
    
    # Constants
    DEFAULT_HOST = "tcp:sql-stat.iaf.local"
    DEFAULT_ODBC_DRIVER = "SQL Server"
    DEFAULT_CATALOG = "Arbetsdatabas"
    DEFAULT_MODELS_PATH =  ".\\src\\IAFautoclass\\model\\"
    DEFAULT_TRAIN_OPTION = "Train a new model!"
    
    # Constructor
    def __init__(self):
        
        # Some data elements might get lost unless we lock a few callback functions
        self.lock_observe_1 = False
        
        # Welcoming message label
        self.welcome = widgets.Label(
            value = "*** Welcome to IAF automatic classification! ***"
        )
        
        # Project element data element
        self.project = widgets.Text(
            value = 'default',
            placeholder = 'Project name',
            description = 'Project name:',
            disabled = False
        )
        
        # A helper for SQL Server communications
        self.sqlHelper = \
            sql.IAFSqlHelper( driver = self.DEFAULT_ODBC_DRIVER, host = self.DEFAULT_HOST, \
                              catalog = self.DEFAULT_CATALOG, trusted_connection = True )
        try:
            self.conn = self.sqlHelper.connect()
            print("GUI class successfully connected to SQL Server!")
        except Exception as ex:
            sys.exit("GUI class could not connect to SQL Server: {0}".format(str(ex)))
        
        # Database dropdown list
        self.database_dropdown = widgets.Dropdown(
            options = ['N/A'],
            value = 'N/A',
            description = 'Databases:',
        )
        
        # Update databases list
        self.update_databases()
        
        # Tables dropdown list
        self.tables_dropdown = widgets.Dropdown(
            options = ['N/A'],
            value = 'N/A',
            description = 'Tables:',
            disabled = True,
        )
        
        # Models dropdown list
        self.models_dropdown = widgets.Dropdown(
            options = ['N/A'],
            value = 'N/A',
            description = 'Models:',
            disabled = True,
        )
        
        # Radiobuttons for choosing the class column
        self.class_column = widgets.RadioButtons(
            options = ['N/A'],
            value = 'N/A',
            description = 'Class:',
            disabled = True,
        )
    
        # Radiobuttons for choosing the unique id column
        self.id_column = widgets.RadioButtons(
            options = ['N/A'],
            value = 'N/A',
            description = 'Unique id:',
            disabled = True,
        )

        # Multiple select for choosing the data columns
        self.data_columns = widgets.SelectMultiple(
            options = ['N/A'],
            value = [],
            description = 'Data:',
            disabled = True,
        )
        
        # Multiple select for picking text columns of the available data columns
        self.text_columns = widgets.SelectMultiple(
            options = ['N/A'],
            value = [],
            description='Text data:',
            disabled = True,
        )
        
         # Continuation message label
        self.continuation = widgets.Label(
            value = "",
        )
        
        # Button to continue after the basic settings was set
        self.continuation_button = widgets.Button(
            description='Continue',
            disabled=True,
            button_style='success', 
            tooltip='Continue with the process using these settings',
            icon='check' 
        )
        
        self.train_checkbox = widgets.Checkbox(
            value = False,
            description = 'Train',
            disabled = True,
            indent = True
        )
        
        self.predict_checkbox = widgets.Checkbox(
            value = False,
            description = 'Predict',
            disabled = True,
            indent = True
        )
        
        self.mispredicted_checkbox = widgets.Checkbox(
            value = False,
            description = 'Display mispredicted',
            disabled = True,
            indent = True
        )
                
        self.checkboxes_form = widgets.Box( 
            [widgets.Label(value='Mode:'), self.train_checkbox, self.predict_checkbox, self.mispredicted_checkbox ], 
            layout = widgets.Layout(
                display='flex',
                flex_flow='row',
                border='solid 0px',
                align_items='stretch',
                width='auto',
            ),
        )
        
    # Destructor
    def __del__(self):
        pass
    
    # Print the class 
    def __str__(self):
        return str(type(self))
    
    # Internal methods used to populate or update widget settings
    def update_databases(self, *args):
        query = "SELECT name FROM sys.databases"
        try:
            self.sqlHelper.execute_query(query, get_data = True)
        except Exception as ex:
            sys.exit("Query \"{0}\" failed: {1}".format(query,str(ex)))
        data = self.sqlHelper.read_all_data()
        database_list = [''] + [database[0] for database in data]
        self.database_dropdown.options = database_list
        self.database_dropdown.value = database_list[0]
        
        self.database_dropdown.observe(self.update_tables)
    
    def update_tables(self, *args):
        if not self.lock_observe_1:
            if self.database_dropdown.value != self.DEFAULT_CATALOG:
                self.sqlHelper.disconnect()
                self.sqlHelper = sql.IAFSqlHelper( driver = self.DEFAULT_ODBC_DRIVER, \
                                                   host = self.DEFAULT_HOST, \
                                                   catalog = self.database_dropdown.value, \
                                                   trusted_connection = True )
            try:
                self.conn = self.sqlHelper.connect()
            except Exception as ex:
                sys.exit("GUI class could not connect to SQL Server: {0}".format(str(ex)))
            query = "SELECT TABLE_SCHEMA,TABLE_NAME FROM INFORMATION_SCHEMA.TABLES ORDER BY TABLE_SCHEMA,TABLE_NAME"
            self.sqlHelper.execute_query(query, get_data = True)
            tables = self.sqlHelper.read_all_data()
            tables_list = ['Pick a table'] + [table[0]+'.'+table[1] for table in tables]
            self.tables_dropdown.options = tables_list
            self.tables_dropdown.value = tables_list[0]
            self.tables_dropdown.disabled = False

            self.tables_dropdown.observe(self.update_models)
        
    def update_models(self, *args):
        if not self.lock_observe_1:
            models_list = ['Pick a model'] + [self.DEFAULT_TRAIN_OPTION] + \
                [file for file in os.listdir(self.DEFAULT_MODELS_PATH)]
            self.models_dropdown.options = models_list
            self.models_dropdown.value = models_list[0]
            self.models_dropdown.disabled = False

            self.models_dropdown.observe(self.use_old_model_or_train_new)
        
    def use_old_model_or_train_new(self, *args):
        if self.models_dropdown.value == self.DEFAULT_TRAIN_OPTION:
            self.update_class_id_data_columns()
            self.continuation_button.disabled = True
        else:
            self.class_column.disabled = True
            self.id_column.disabled = True 
            self.data_columns.disabled = True
            self.text_columns.disabled = True
            self.continuation_button.on_click(callback=self.continuation_button_was_clicked)
            self.continuation_button.disabled = False
        
    def update_class_id_data_columns(self, *args):
        query = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_CATALOG = " \
            + "\'" + self.database_dropdown.value + "\' AND CONCAT(CONCAT(TABLE_SCHEMA,'.'),TABLE_NAME) = " \
            + "\'" + self.tables_dropdown.value + "\'"
        self.sqlHelper.execute_query(query, get_data = True)
        columns = self.sqlHelper.read_all_data()
        columns_list = [column[0] for column in columns]
        
        if len(columns_list) >= 3:
            self.class_column.options = columns_list
            self.class_column.value = columns_list[0]
            self.class_column.disabled = False
            self.class_column.observe(self.update_id_and_data_columns)

            self.id_column.options = columns_list[1:]
            self.id_column.value = columns_list[1]
            self.id_column.disabled = False
            self.id_column.observe(self.update_data_columns)

            self.data_columns.options = columns_list[2:]
            self.data_columns.value = []
            self.data_columns.disabled = False
            self.data_columns.observe(self.update_text_columns_and_enable_button)
        else:
            sys.exit("Selected data table has to few columns. Minimum is three (3).")
            
    def update_id_and_data_columns(self, *args):
        self.id_column.options = \
            [option for option in self.class_column.options if option != self.class_column.value]
        self.update_data_columns(*args)

    def update_data_columns(self, *args):
        self.data_columns.options = \
            [option for option in self.id_column.options if option != self.id_column.value]
            
    def update_text_columns_and_enable_button(self, *args):
        self.text_columns.options = self.data_columns.value
        self.text_columns.value = []
        self.text_columns.disabled = False
        
        if len(self.data_columns.value) >= 1:
            self.continuation_button.disabled = False
            self.continuation_button.on_click(callback=self.continuation_button_was_clicked)
        else:
            self.continuation_button.disabled = True
                
    def update_continuation_label(self, *args):
        self.continuation.value = \
            "You picked the following settings: \n\t Class column: {} \n\t Unique id: {} \n\t Data column: {}\n".\
            format(self.class_column.value,self.id_column.value, ', '.join(self.data_columns.value)) + \
            "\n" + "Now specify what data columns that contain text (the others will assume to be numerical):"
        self.continuation.disabled = False
        
    def continuation_button_was_clicked(self, *args):
        self.lock_observe_1 = True
        if self.models_dropdown.value != self.DEFAULT_TRAIN_OPTION:
            self.train_checkbox.value = False
            self.train_checkbox.disabled = True
            self.predict_checkbox.value = True
            self.predict_checkbox.disabled = True
            self.mispredicted_checkbox.value = False
            self.mispredicted_checkbox.disabled = True
        else:
            self.train_checkbox.value = True
            self.train_checkbox.disabled = False
            self.predict_checkbox.value = False
            self.predict_checkbox.disabled = False
            self.mispredicted_checkbox.value = False
            self.mispredicted_checkbox.disabled = False
        self.project.disabled = True
        self.database_dropdown.disabled = True
        self.tables_dropdown.disabled = True
        self.models_dropdown.disabled = True
        self.class_column.disabled = True
        self.id_column.disabled = True
        self.data_columns.disabled = True
        self.text_columns.disabled = True

# Create GUI object
gui = IAFautoclass_GUI()

GUI class successfully connected to SQL Server!


In [4]:
# Welcome message
display(gui.welcome)

# Project name
display(gui.project)

# Database dropdown list
display(gui.database_dropdown)

# Tables dropdown list
display(gui.tables_dropdown)

# Models dropdown list
display(gui.models_dropdown)

# Choosing class column, unique id column and data columns
display(gui.class_column)
display(gui.id_column)
display(gui.data_columns)
display(gui.continuation)
display(gui.text_columns)

# Press a button to continue
display(gui.continuation_button)

display(gui.checkboxes_form)

Label(value='*** Welcome to IAF automatic classification! ***')

Text(value='default', description='Project name:', placeholder='Project name')

Dropdown(description='Databases:', options=('', 'master', 'tempdb', 'model', 'msdb', 'AF', 'astat3_gallring', …

Dropdown(description='Tables:', disabled=True, options=('N/A',), value='N/A')

Dropdown(description='Models:', disabled=True, options=('N/A',), value='N/A')

...

SelectMultiple(description='Data:', disabled=True, options=('N/A',), value=())

Label(value='')

SelectMultiple(description='Text data:', disabled=True, options=('N/A',), value=())

Button(button_style='success', description='Continue', disabled=True, icon='check', style=ButtonStyle(), toolt…

Box(children=(Label(value='Mode:'), Checkbox(value=False, description='Train', disabled=True), Checkbox(value=…